# Implementare Simplă a Perceptronului
_de Mihai Dan Nadăș (mihai.nadas@ubbcluj.ro), Ianuarie 2025_

Acest notebook implementează o versiune a perceptronului așa cum a fost introdus în lucrarea din 1958 a lui Frank Rosenblatt, "The Perceptron: A Probabilistic Model for Information Storage and Organization in the Brain".

Ne vom concentra pe concepte matematice de bază și vom minimiza utilizarea algebrei liniare, evitând operațiile cu vectori și matrici acolo unde este posibil.

## Obiectiv

Obiectivul nostru este să antrenăm un model cu două greutăți, $w_{1}$ și $w_{2}$, corespunzătoare coordonatelor $x$ și $y$ ale unui punct $(x, y)$, plus o bias $b$. Acest lucru se bazează pe adaptarea ecuației algebrice $y = mx + c$, care reprezintă forma pantei-interceptului a unei drepte.

Modelul va aborda o sarcină simplă de clasificare cu un set de date liniar separabil, generat conform următoarei funcții:

Dat un număr întreg $x$:

$
f: \mathbb{N} \to \mathbb{N}, \quad f(x) =
\begin{cases}
x, & \text{dacă } x \bmod 2 = 0, \\
2x, & \text{dacă } x \bmod 2 = 1.
\end{cases}
$

Această funcție clasifică numerele astfel: dacă $x$ este par, rămâne neschimbat; dacă este impar, se dublează. Această transformare va ajuta la clasificarea numerelor pe baza parității lor.

## Set de date
Vom genera un set de date folosind Biblioteca Standard Python.

In [ ]:
import random


def generate_dataset(num_items=20, start=0, stop=100):
    random.seed(42)
    dataset = []
    x1_values = set()
    while len(dataset) < num_items:
        x1 = random.randint(start, stop)
        if x1 in x1_values:
            continue
        x1_values.add(x1)
        x2 = x1 if x1 % 2 == 0 else 2 * x1
        y = (
            0 if x1 == x2 else 1
        )  # (x1, x2) este etichetat ca Clasa 0 dacă x1 este par, și Clasa 1 altfel
        dataset.append((x1, x2, y))
    return dataset


dataset = generate_dataset()

# acum să împărțim setul de date în seturi de antrenament și test
train_ratio = 0.8
num_train = int(len(dataset) * train_ratio)
dataset_train, dataset_test = dataset[:num_train], dataset[num_train:]
print(f"Set de antrenament (n={len(dataset_train)}): {dataset_train}")
print(f"Set de testare (n={len(dataset_test)}: {dataset_test}")

## Reprezentare Vizuală

În această secțiune, vom vizualiza seturile de date de antrenament și testare folosind _Matplotlib_ și _pandas_. Această vizualizare ne permite să vedem clar separabilitatea claselor, care este o parte esențială a înțelegerii modului în care va funcționa algoritmul perceptronului. Prin examinarea graficelor, putem identifica frontiera de decizie liniară și evalua cât de bine pot fi clasificate datele.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


def plot_datasets(train_dataset, test_dataset):
    # Combină seturile de date într-un DataFrame pentru gestionare mai ușoară
    train_df = pd.DataFrame(train_dataset, columns=["x1", "x2", "class"])
    train_df["set"] = "Train"

    test_df = pd.DataFrame(test_dataset, columns=["x1", "x2", "class"])
    test_df["set"] = "Test"

    combined_df = pd.concat([train_df, test_df], ignore_index=True)

    # Definire culori și markere
    colors = {0: "blue", 1: "red"}
    markers = {"Train": "o", "Test": "x"}

    # Plotare fiecare grup folosind Matplotlib
    fig, ax = plt.subplots()
    for (dataset, cls), group in combined_df.groupby(["set", "class"]):
        ax.scatter(
            group["x1"],
            group["x2"],
            color=colors[cls],
            label=f"{dataset} Dataset, Class {cls}",
            s=30,
            marker=markers[dataset],
        )

    # Gestionare legendă și etichete
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), title="Dataset and Class", loc="best")
    ax.set_xlabel("x1")
    ax.set_ylabel("x2")
    ax.set_title("Training and Test Datasets")
    ax.grid(True)


plot_datasets(dataset_train, dataset_test)
plt.show()

Definirea unui Clasificator Liniar

Cu setul nostru de date pregătit, ne îndreptăm acum spre fundamentul matematic care permite modelului nostru să clasifice o intrare $x_{1}$ și $x_{2}$ ca aparținând claselor $0$ sau $1$. Funcția de clasificare este definită astfel:

$
c: \mathbb{N} \times \mathbb{N} \to \{0,1\}, \quad
c(x_{1}, x_{2}) =
\begin{cases} 
1, & \text{dacă } (x_{1}, x_{2}) \in \text{Clasa 1}, \\
0, & \text{dacă } (x_{1}, x_{2}) \in \text{Clasa 0}.
\end{cases}
$

Această clasificare se realizează utilizând reprezentarea algebrică a unei linii într-un sistem de coordonate carteziene, formulată astfel:

$
z(x) = w_{1}x_{1} + w_{2}x_{2} + c,
$

unde:
- $w_{1}$ și $w_{2}$ sunt greutățile care determină panta, reprezentând unghiul liniei în raport cu axa $x$,
- $c$ este biasul (sau interceptul), indicând unde intersectează linia axa $y$.

Din graficul de mai sus, este evident că cele două clase sunt separabile liniar, justificând utilizarea unei frontiere de decizie liniare. Greutățile $w_{1}$ și $w_{2}$ sunt ajustate prin antrenament folosind algoritmul Perceptron al lui Rosenblatt pentru a realiza eficient această separare.

Pentru o înțelegere mai clară, iată cum ar apărea o linie definită de $w_{1} = 1$, $w_{2} = 0.5$ și $c = 0$ pe graficul nostru anterior, ilustrând separarea între clase.

In [ ]:
zx = lambda x1, x2, w1, w2, c: w1 * x1 + w2 * x2 + c


def plot_zx(w1, w2, c):
    x2 = lambda x1: (
        (-w1 * x1 - c) / w2 if w2 != 0 else -c / w1 if w1 != 0 else c
    )  # acest lucru se datorează faptului că ecuația liniei este w1*x1 + w2*x2 + c = 0, prin urmare x2 = (-w1*x1 - c) / w2
    x1_values = range(0, 101)
    x2_values = [x2(x1) for x1 in x1_values]
    plt.plot(x1_values, x2_values, label=f"{w1}x1+{w2}x2+{c}=0")
    plt.legend(loc="best")


def plot_datasets_and_zx(w1, w2, c):
    plot_datasets(dataset_train, dataset_test)
    plot_zx(w1, w2, c)


plot_datasets_and_zx(-1.5, 1.1, -10)

Acum, este clar că în această configurație, punctele de date sunt separate eficient. Cu toate acestea, există și alte configurații pentru $w_{1}$, $w_{2}$ și $c$ care duc la un exemplu mai puțin eficient. De exemplu, când $w_{1} = 0.1$, $w_{2} = 0.1$ și $c = 0.5$, obținem o limită de separare mai puțin optimă.

In [ ]:
plot_datasets_and_zx(0.1, 0.1, 0.5)

În acest caz particular, funcția $z$ nu este eficientă în clasificarea punctelor de date, indicând că valorile curente ale ponderilor și ale bias-ului nu sunt potrivite pentru sarcină.

## Evaluarea Performanței Clasificatorului

Cu limita de decizie $z = w_{1}x_{1} + w_{2}x_{2} + c$ definită, următorul nostru pas este să evaluăm eficiența sa folosind o metrică numită _acuratețe_. Aceasta ne va oferi o măsură cuantificabilă despre cât de bine funcționează clasificatorul nostru pe setul de date.

### Definirea Funcției Clasificatorului

Înainte de a-i evalua performanța, să definim clar clasificatorul nostru. Acesta poate fi exprimat astfel:

$c: \mathbb{R}^2 \to \{0,1\}, \quad
c(x_{1}, x_{2}) = 
\begin{cases} 
1, & \text{dacă } z(x_{1}, x_{2}) \geq 0, \\
0, & \text{dacă } z(x_{1}, x_{2}) < 0.
\end{cases}
$

Aceasta înseamnă că orice punct $(x_{1}, x_{2})$ care se află deasupra sau pe limita de decizie, calculată prin ecuația noastră liniară, va fi clasificat ca 1. În schimb, punctele de dedesubt vor fi clasificate ca 0.

Să implementăm acest clasificator în cod, și apoi vom continua să discutăm și să-i evaluăm performanța în mai mult detaliu.

In [ ]:
cx = lambda x1, x2, w1, w2, c: 1 if zx(x1, x2, w1, w2, c) >= 0 else 0


def accuracy(dataset, w1, w2, c):
    print(f"Calcul calculul acurateții pe setul de antrenament folosind w1={w1}, w2={w2}, c={c}")
    correct = 0
    for x1, x2, y in dataset:
        if y == cx(x1, x2, w1, w2, c):
            correct += 1
    print(
        f"Acuratețea rezultată: {correct}/{len(dataset)}, sau {correct/len(dataset)*100:.2f}%"
    )
    return correct / len(dataset)


# Aplicarea funcției de acuratețe la setul de antrenament folosind cele două seturi de greutăți și bias așa cum este arătat mai sus, în primul exemplu
accuracy(dataset_train, -1.5, 1.1, -10)

# Aplicarea funcției de acuratețe la setul de antrenament folosind două seturi de greutăți și bias așa cum este arătat mai sus, în al doilea exemplu
accuracy(dataset_train, 0.1, 0.1, 0.5)

### Discuție despre Acuratețe

După cum s-a arătat anterior, modificarea valorilor de greutăți și biasuri duce la rezultate diferite în ceea ce privește acuratețea. Acest lucru se datorează faptului că greutățile și biasurile definesc frontiera de decizie, care influențează acuratețea clasificării pe baza modului în care separă clasele din setul de date. Prin urmare, provocarea este de a descoperi valorile "optime" pentru acești parametri. Acest lucru se realizează printr-un proces cunoscut sub numele de _antrenare a modelului_.

## Antrenarea Modelului

Vom antrena acum modelul nostru folosind cunoștințele din analiza noastră. Aceasta implică o serie de iterări pentru a ajusta valorile greutăților și biasurilor până când se obține un nivel satisfăcător de acuratețe. Această metodă iterativă ne ajută să găsim cea mai bună combinație de parametri adaptată setului nostru de date și sarcinii de clasificare.

### Concepte Importante în Antrenarea Modelului

În cadrul antrenării modelului, anumite concepte sunt cruciale:

- **Epoci:** Acest termen se referă la o trecere completă prin întregul set de date folosit pentru antrenare. Pot fi necesare mai multe epoci pentru a rafina modelul.
- **Rata de Învățare:** Este pasul folosit la actualizarea parametrilor modelului. Aceasta determină cât de repede sau încet învață modelul.
- **Actualizări de Greutăți:** În timpul antrenării, greutățile sunt ajustate pe baza erorii predicțiilor pentru a minimiza divergența față de etichetele adevărate.

Aceste concepte joacă un rol esențial în antrenarea eficientă a modelului pentru a se asigura că generalizează bine pentru date noi. Prin iterare cu epoci și rate de învățare diferite, putem optimiza frontiera de decizie pentru rezultate mai bune în clasificare.

In [ ]:
# Mai întâi, să inițializăm greutățile și bias-ul la zero
w1, w2, c = 0, 0, 0

# Să definim acum rata de învățare
learning_rate = 0.1

# Să definim acum numărul de epoci
num_epochs = 100

# Creăm un DataFrame pentru a stoca detaliile epocilor
epoch_details = pd.DataFrame(columns=["epocă", "x1", "x2", "y", "z", "y_hat", "w1", "w2", "c"])

# Să începem acum bucla de antrenament
for epoch in range(num_epochs):
    print(f"Epocă {epoch+1}")
    for x1, x2, y in dataset_train:
        z = zx(x1, x2, w1, w2, c)
        y_hat = 1 if z >= 0 else 0
        w1 += learning_rate * (y - y_hat) * x1
        w2 += learning_rate * (y - y_hat) * x2
        c += learning_rate * (y - y_hat)
        print(f"  x1={x1}, x2={x2}, y={y}, z={z:.2f}, y_hat={y_hat}, w1={w1:.2f}, w2={w2:.2f}, c={c:.2f}")
        # Adăugăm detaliile la DataFrame
        epoch_details = epoch_details.concat({
            "epocă": epoch + 1,
            "x1": x1,
            "x2": x2,
            "y": y,
            "z": z,
            "y_hat": y_hat,
            "w1": w1,
            "w2": w2,
            "c": c
        }, ignore_index=True)

# Afișăm DataFrame-ul
epoch_details.head()